In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install customized_konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 881.5/881.5 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 36.2 MB/s eta 0:00:00


In [ ]:
!pip install tensorflow_addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 6.1 MB/s eta 0:00:00


In [ ]:
import os
import pandas as pd
import numpy as np
import re
from konlpy.tag import Okt, Kkma
from tqdm import tqdm
from ckonlpy.tag import Twitter
import urllib.request
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow_addons as tfa
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, \
                            roc_auc_score, confusion_matrix, classification_report, \
                            matthews_corrcoef, cohen_kappa_score, log_loss

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [ ]:
# 성능평가

In [ ]:
from transformers import TFBertForSequenceClassification
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import custom_object_scope

# 모델을 로드할때 해당layer가 제대로 인식되도록 사용자 지정 객체 스코프에 등록 , 로드
with custom_object_scope({'TFBertForSequenceClassification': TFBertForSequenceClassification}):
    loaded_model = load_model('/content/drive/MyDrive/playdataproject/model/best_model(2).h5')


In [ ]:
loaded_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_word_ids (InputLayer  [(None, 128)]                0         []                            
 )                                                                                                
                                                                                                  
 input_masks (InputLayer)    [(None, 128)]                0         []                            
                                                                                                  
 input_segment (InputLayer)  [(None, 128)]                0         []                            
                                                                                                  
 tf_bert_for_sequence_class  TFSequenceClassifierOutput   1106188   ['input_word_ids[0][0]',  

In [ ]:
def convert_data(X_data):
    # BERT 입력으로 들어가는 token, mask, segment, target 저장용 리스트
    tokens, masks, segments = [], [], []

    for X in tqdm(X_data):
        # token: 입력 문장 토큰화
        token = tokenizer.encode(X, truncation = True, padding = 'max_length', max_length = MAX_SEQ_LEN)

        # Mask: 토큰화한 문장 내 패딩이 아닌 경우 1, 패딩인 경우 0으로 초기화
        num_zeros = token.count(0)
        mask = [1] * (MAX_SEQ_LEN - num_zeros) + [0] * num_zeros

        # segment: 문장 전후관계 구분: 오직 한 문장이므로 모두 0으로 초기화
        segment = [0]*MAX_SEQ_LEN

        tokens.append(token)
        masks.append(mask)
        segments.append(segment)

    # numpy array로 저장
    tokens = np.array(tokens)
    masks = np.array(masks)
    segments = np.array(segments)

    return [tokens, masks, segments]

In [ ]:
def predict_sentence(input_sentence):
    # 입력을 BERT 모델에 맞게 변환
    input_data = convert_data([input_sentence])

    input_tokens, input_masks, input_segments = input_data
    test = [input_tokens , input_masks, input_segments]
    # 텐서로 변환
    # input_tokens = tf.constant(input_tokens)
    # input_masks = tf.constant(input_masks)
    # input_segments = tf.constant(input_segments)

    # 모델 예측
    predictions = loaded_model.predict(test)

    # 예측된 클래스 및 확률 출력
    predicted_class = np.argmax(predictions, axis=-1)[0]

    print(f"예측된 클래스: {predicted_class}")
    print(f"클래스별 확률: {predictions}")
    return predicted_class

In [ ]:
tokenizer = BertTokenizer.from_pretrained("klue/bert-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

In [ ]:
MAX_SEQ_LEN = 128

In [ ]:
def count_pos_neg (location) : #매장별로 긍정 부정 비율 카운트 함수
  data = df[df['LOCATION'] == location]
  reviews = data['Review']
  reviews.dropna(inplace=True)
  pos = 0
  neg = 0
  for review in reviews :
    result = predict_sentence(review)
    if result == 0:
      neg+=1
    else :
      pos+=1
  neg_percent = (neg / len(reviews) * 100.0)
  pos_percent = (pos / len(reviews) * 100.0)
  return [location, neg, pos, neg_percent, pos_percent]

In [ ]:
df_secho = pd.read_excel("/content/drive/MyDrive/playdataproject/data/2024-02-15 '서초구'.xlsx")
df_gangnam = pd.read_excel("/content/drive/MyDrive/playdataproject/data/2024-02-15 '강남구'.xlsx")
df_songpa = pd.read_excel("/content/drive/MyDrive/playdataproject/data/2024-02-15 '송파구'.xlsx")
df_list = [df_secho, df_gangnam, df_songpa]
df = pd.concat(df_list, axis = 0)
df

,LOCATION,ID,Review,Date
0,동대문엽기떡볶이 강남역점,찰떡님,강남역 동대문 엽떡. 마라맛 엽떡을 아직 먹어보지 못해 방문해보았다. 마라향이 은은...,2024년 1월 8일 월요일
1,동대문엽기떡볶이 강남역점,n0w25,"입구찾기 어려웠던,,🥲 마라로제 주문했는데 콘마요 없어서 못먹구 꿔바로우도 없고,,...",2024년 1월 28일 일요일
2,동대문엽기떡볶이 강남역점,HJ811,강남역점은 분모자 추가 안돼요,2024년 1월 20일 토요일
3,동대문엽기떡볶이 강남역점,맛도리체이서,마라엽떡 맛있음😋,2023년 11월 23일 목요일
4,동대문엽기떡볶이 강남역점,아하하하하하잉,올해 마지막 엽떡! 행복해따,2023년 12월 22일 금요일
...,...,...,...,...
1508,동대문엽기떡볶이 문정점,rla****,맛있어요,2020년 3월 19일 목요일
1509,동대문엽기떡볶이 문정점,쿠키순삭,너무 맛나요,2020년 3월 5일 목요일
1510,동대문엽기떡볶이 문정점,kkorose,여기는 맛부터 그냥 다 최악인듯 .. 오리지널 시켯는데 밍밍한맛? 매운것도아닌 밍...,2020년 1월 10일 금요일
1511,동대문엽기떡볶이 문정점,베리 페리,매움의 단계가 가끔 일정하지 않음,2019년 12월 1일 일요일


In [ ]:
loc_list = df['LOCATION'].unique()

In [ ]:
columns = ['LOCATION', 'neg', 'pos', 'neg_percent', 'pos_percent']
result_df = pd.DataFrame(columns = columns)
result_df

,LOCATION,neg,pos,neg_percent,pos_percent


In [ ]:
for loc in loc_list:
  new_row = pd.Series(count_pos_neg(loc), index=columns)
  result_df = result_df.append(new_row, ignore_index=True)

<ipython-input-56-d34847f3544f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews.dropna(inplace=True)
100%|██████████| 1/1 [00:00<00:00, 356.84it/s]

1/1 [==============================] - 0s 63ms/step


예측된 클래스: 1
클래스별 확률: [[0.24329393 0.75670606]]


100%|██████████| 1/1 [00:00<00:00, 944.66it/s]


1/1 [==============================] - 0s 57ms/step
예측된 클래스: 0
클래스별 확률: [[0.7503018  0.24969824]]


...

100%|██████████| 1/1 [00:00<00:00, 1649.35it/s]


1/1 [==============================] - 0s 91ms/step
예측된 클래스: 0
클래스별 확률: [[0.75026214 0.2497379 ]]


100%|██████████| 1/1 [00:00<00:00, 922.23it/s]


1/1 [==============================] - 0s 77ms/step
예측된 클래스: 1
클래스별 확률: [[0.24299155 0.75700843]]


<ipython-input-58-42d5f8e3ee35>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(new_row, ignore_index=True)


In [ ]:
result_df.to_excel('/content/drive/MyDrive/playdbataproject/data/매장별리뷰비율.xlsx')

OSError: Cannot save file into a non-existent directory: '/content/drive/MyDrive/playdbataproject/data'

In [ ]:
result_df

,LOCATION,neg,pos,neg_percent,pos_percent
0,동대문엽기떡볶이 강남역점,37,260,12.457912,87.542088
1,동대문엽기떡볶이 방배점,26,155,14.364641,85.635359
2,동대문엽기떡볶이 교대점,39,308,11.239193,88.760807
3,동대문엽기떡볶이 신반포점,20,127,13.605442,86.394558
4,동대문엽기떡볶이 신사역점,16,97,14.159292,85.840708
5,동대문엽기떡볶이 개포점,18,156,10.344828,89.655172
6,동대문엽기떡볶이 양재우면점,21,211,9.051724,90.948276
7,동대문엽기떡볶이 구반포점,11,94,10.476190,89.523810
8,동대문엽기떡볶이 반포점,0,2,0.000000,100.000000
9,동대문엽기떡볶이 강남구청역점,20,127,13.605442,86.394558


In [ ]:
print(f'긍정: {"%.2f%%" % (len(pos_result)/ len(data) * 100.0)}')
print(f'부정: {"%.2f%%" % (len(neg_result)/ len(data) * 100.0)}')
print(len(pos_result))

긍정: 77.08%
부정: 22.92%
390
